In [1]:
#import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle
from sklearn import svm

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [2]:
filename_train ='/home/octo/Dropbox'+ '/SPY9Apr.csv'

In [7]:
# loading csv file
def get_csv_pd(path):
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    return spy_pd
def BA(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df=df.dropna()
    return df
def processing(df):
    df['Open']=(df.askPrice+df.bidPrice)/2
    df['Close']=df.Open.shift(-1)
    df['High']=df.askPrice.rolling(6).max()
    df['Low']=df.bidPrice.rolling(6).min()
    df['Volume']=df.askSize+df.bidSize
    df['vwap']=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df=df.dropna()
    return df
def tech_ind(data):
    data['km']=ml.kalman_ma(data)
    data['rsi']=talib.RSI(np.array(data.Close.astype('float64')),timeperiod=6)
    data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype  ('float64')),timeperiod=6)
    data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=6)# mom consists 0
    return data


In [13]:
data=get_csv_pd(filename_train)
data=BA(data)
data=processing(data)
data=tech_ind(data)
data=data.drop(['Stock'], axis=1)
data=(data-data.rolling(6).mean())
data['UD']=np.where(np.logical_and(data.Close.diff(6)>0.005,data.askPrice>data.km*1.001),1,np.where(np.logical_and(data.Close.diff(6)<-0.005,data.bidPrice<data.km*0.999),-1,0))
data['U']=np.where(np.logical_and(data.Close.diff(6)>0.005,data.askPrice>data.km*1.001),1,0)
data['D']=np.where(np.logical_and(data.Close.diff(6)<-0.005,data.bidPrice<data.km*0.999),-1,0)
data=data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom','UD','U','D']]
data=data.astype('float64')
data=data.dropna()

In [14]:
data=data[28000:45010]

In [ ]:
#data.head()

In [ ]:
#data.tail()

In [15]:
X=data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']]
Y=data.UD

## predicting state

In [16]:
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])
data['h_state']=predictions_hmm

In [17]:
np.argmin(np.diag(hmm_loaded.transmat_))

2

In [18]:
data.groupby('h_state').size()

h_state
0    1411
1    8844
2    6755
dtype: int64

## Bayes

### Up

In [24]:
filename_nb = 'nbU.sav'
nb_loaded = pickle.load(open(filename_nb, 'rb'))
predictions_nb = nb_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])

In [25]:
data['prNBu']=predictions_nb

In [26]:
data.groupby('prNBu').size()

prNBu
0.00000    11404
1.00000     5606
dtype: int64

### Dn

In [27]:
filename_nb = 'nbD.sav'
nb_loaded = pickle.load(open(filename_nb, 'rb'))
predictions_nb = nb_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])
data['prNBd']=predictions_nb

In [28]:
data.groupby('prNBd').size()

prNBd
-1.00000     6593
0.00000     10417
dtype: int64

In [52]:
data['prNB']=data.prNBu+data.prNBd

## outlier

In [35]:
filename_outlr = 'outlr.sav'
outlr_loaded = pickle.load(open(filename_outlr, 'rb'))
predictions_outlr = outlr_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])
data['outlr']=predictions_outlr

In [36]:
data.groupby('outlr').size()

outlr
-1      765
 1    16245
dtype: int64

## Predicting UD by RF

In [38]:
filename_rf = 'rf.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])

In [39]:
data['prUD']=predictions_rf

In [40]:
rf_loaded.score(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']],data.UD)

0.97554379776602

In [41]:
data.groupby('prUD').size()

prUD
-1.00000     3232
0.00000     11496
1.00000      2282
dtype: int64

## Predicting UD by GB

In [44]:
filename_gb = 'gb.sav'
gb_loaded = pickle.load(open(filename_gb, 'rb'))
predictions_gb = gb_loaded.predict(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']])

In [45]:
data['prUDg']=predictions_gb

In [46]:
gb_loaded.score(data[['askPrice','bidPrice','High','Low','Volume','vwap','rsi','atr','mom']],data.UD)

0.9516754850088184

## Considering h_state and outlier data as  NAP. 

In [108]:
st=data.h_state==np.argmin(np.diag(hmm_loaded.transmat_))

In [125]:
data['UD']=data.prUD+data.prNB
data['UD']=np.where(data.UD==2,1,np.where(data.UD==-2,-1,0))
data['trade']=np.where(np.logical_or(np.logical_and(data.outlr==-1,data.UD==1),np.logical_and(st==False,data.UD==1)),1,np.where(np.logical_or(np.logical_and(data.outlr==-1,data.UD==-1),np.logical_and(st==False,data.UD==-1)),-1,0))

In [126]:
data.groupby('UD').size()

UD
-1     3027
 0    12018
 1     1965
dtype: int64

In [127]:
data.groupby('trade').size()

trade
-1      621
 0    14437
 1     1952
dtype: int64

In [128]:
pd.crosstab(index=data.outlr,columns=data.UD) 

UD,-1,0,1
outlr,,,
-1,31,710,24
1,2996,11308,1941


In [129]:
pd.crosstab(index=st,columns=data.UD)

UD,-1,0,1
h_state,,,
False,604,7699,1952
True,2423,4319,13
